In [ ]:
pip install tensorflow keras music21

# Albeniz

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/albeniz").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_albeniz.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("albenizgenerator_model.h5")
    gan.discriminator.save("albenizdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt

def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/albeniz'  
notes = get_notes(midi_folder)


In [ ]:
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian."""
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    """Builds the VAE model and also returns the encoder."""
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    # Instantiate the decoder model
    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    # Loss function: binary crossentropy and KL divergence
    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    # Return both the VAE and encoder models
    return vae, encoder

latent_dim = 1000  # This should match the GAN's latent dimension
sequence_length = 100  # This should match the length of your music sequences

# Use the modified function to build the VAE and get the encoder
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)

# Save the encoder model
vae_encoder.save('albeniz_vae_encoder.h5')


In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)


In [ ]:
vae.save('/kaggle/working/albeniz_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/albenizgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'albeniz_generated_music_{}'.format(i))


# Bach

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/bach").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_bach.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("bachgenerator_model.h5")
    gan.discriminator.save("bachdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/bach'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('bach_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/bach_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/bachgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'bach_generated_music_{}'.format(i))


# Balakir

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/balakir").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_balakir.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("balakirgenerator_model.h5")
    gan.discriminator.save("balakirdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/balakir'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('balakir_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/balakir_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/balakirgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'balakir_generated_music_{}'.format(i))


# Beeth

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/beeth").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_beeth.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("beethgenerator_model.h5")
    gan.discriminator.save("beethdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/beeth'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('beeth_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/beeth_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/beethgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'beeth_generated_music_{}'.format(i))


# Borodin

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/borodin").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_borodin.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("borodingenerator_model.h5")
    gan.discriminator.save("borodindiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/borodin'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('borodin_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/borodin_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/borodingenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'borodin_generated_music_{}'.format(i))


# Brahms

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/brahms").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_brahms.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("brahmsgenerator_model.h5")
    gan.discriminator.save("brahmsdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/brahms'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('brahms_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/brahms_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/brahmsgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'brahms_generated_music_{}'.format(i))


# Burgm

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/burgm").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_burgm.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("burgmgenerator_model.h5")
    gan.discriminator.save("burgmdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/burgm'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('burgm_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/burgm_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/burgmgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'burgm_generated_music_{}'.format(i))


# Chopin

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/chopin").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_chopin.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("chopingenerator_model.h5")
    gan.discriminator.save("chopindiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/chopin'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('chopin_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/chopin_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/chopingenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'burgm_generated_music_{}'.format(i))


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/chopingenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'chopin_generated_music_{}'.format(i))


# Debussy

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/debussy").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_debussy.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("debussygenerator_model.h5")
    gan.discriminator.save("debussydiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/debussy'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('debussy_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/debussy_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/debussygenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'debussy_generated_music_{}'.format(i))


# Granados

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/granados").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_granados.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("Granadosgenerator_model.h5")
    gan.discriminator.save("Granadosdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/granados'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('Granados_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/Granados_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/Granadosgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'Granados_generated_music_{}'.format(i))


# Grieg

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/grieg").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_grieg.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("grieggenerator_model.h5")
    gan.discriminator.save("griegdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/grieg'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('grieg_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/grieg_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/grieggenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'grieg_generated_music_{}'.format(i))


# Haydn

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/haydn").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_haydn.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("haydngenerator_model.h5")
    gan.discriminator.save("haydndiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/haydn'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('haydn_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/haydn_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/haydngenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'haydn_generated_music_{}'.format(i))


# liszt

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/liszt").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_liszt.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("lisztgenerator_model.h5")
    gan.discriminator.save("lisztdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/liszt'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('liszt_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/liszt_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/lisztgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'liszt_generated_music_{}'.format(i))


# mendelssohn

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/mendelssohn").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_mendelssohn.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("mendelssohngenerator_model.h5")
    gan.discriminator.save("mendelssohndiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/mendelssohn'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('mendelssohn_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/mendelssohn_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/mendelssohngenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'mendelssohn_generated_music_{}'.format(i))


# mozart

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/mozart").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_mozart.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("mozartgenerator_model.h5")
    gan.discriminator.save("mozartdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/mozart'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('mozart_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/mozart_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/mozartgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'mozart_generated_music_{}'.format(i))


# muss

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/muss").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_muss.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("mussgenerator_model.h5")
    gan.discriminator.save("mussdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/muss'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('muss_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/muss_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/mussgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'muss_generated_music_{}'.format(i))


# schubert

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/schubert").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_schubert.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("schubertgenerator_model.h5")
    gan.discriminator.save("schubertdiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/schubert'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('schubert_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/schubert_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/schubertgenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'schubert_generated_music_{}'.format(i))


# schumann

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/schumann").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_schumann.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("schumanngenerator_model.h5")
    gan.discriminator.save("schumanndiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/schumann'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('schumann_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/schumann_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/schumanngenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'schumann_generated_music_{}'.format(i))


# tschai

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt


SEQUENCE_LENGTH = 100
LATENT_DIMENSION = 1000
BATCH_SIZE = 16
EPOCHS = 10
SAMPLE_INTERVAL = 1

# Define the legacy optimizer
legacy_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("/kaggle/input/7th-april-generated-dataset/augmented_dataset/tschai").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab) / 2) / (float(n_vocab) / 2)
    network_output = to_categorical(network_output, num_classes=n_vocab)  # Use to_categorical from TensorFlow's Keras

    return network_input, network_output  # Add this return statement

  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=legacy_optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(generated_seq)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=legacy_optimizer)
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)
        return Model(seq, validity)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        return Model(noise, seq)
    def train(self, epochs, batch_size=128, sample_interval=50):
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_seqs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real)
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes_mapped = []
        for x in pred_notes:
            index = int(x)
            if index in int_to_note:
                pred_notes_mapped.append(int_to_note[index])
            else:
                pred_notes_mapped.append('C5')         
        create_midi(pred_notes_mapped, 'gan_final')

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_tschai.png', transparent=True)
        plt.close()

if __name__ == '__main__':
    gan = GAN(rows=SEQUENCE_LENGTH)
    gan.train(epochs=EPOCHS, batch_size=BATCH_SIZE, sample_interval=SAMPLE_INTERVAL)

    # Save the generator and discriminator models
    gan.generator.save("tschaigenerator_model.h5")
    gan.discriminator.save("tschaidiscriminator_model.h5")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
from pathlib import Path
import matplotlib.pyplot as plt
def get_notes(midi_folder):
    """ Get all the notes and chords from the midi files in the specified folder """
    notes = []

    # Replace '/path/to/midi/folder' with the path to your folder containing MIDI files
    for file in Path(midi_folder).glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes


In [ ]:
midi_folder = '/kaggle/input/7th-april-generated-dataset/augmented_dataset/tschai'  
notes = get_notes(midi_folder)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim, sequence_length, n_vocab):
    # Encoder model
    inputs = Input(shape=(sequence_length, 1), name='encoder_input')
    x = LSTM(512, return_sequences=True)(inputs)
    x = LSTM(256)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    # Sampling layer
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # Decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = RepeatVector(sequence_length)(latent_inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(512, return_sequences=True)(x)
    outputs = Dense(1, activation='tanh')(x)

    decoder = Model(latent_inputs, outputs, name='decoder')

    # VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')

    reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= sequence_length
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae, encoder
latent_dim = 1000  
sequence_length = 100  
vae, vae_encoder = build_vae(latent_dim, sequence_length, n_vocab)
vae_encoder.save('tschai_vae_encoder.h5')

In [ ]:
vae.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE)
vae.save('/kaggle/working/tschai_vae_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained VAE encoder model
vae_encoder = vae_encoder

def train_gan_with_vae(gan, vae_encoder, epochs, batch_size=128, sample_interval=50):
    # Load and convert the data
    notes = get_notes(midi_folder)  # This function needs to be defined to load your MIDI data
    n_vocab = len(set(notes))
    X_train, _ = prepare_sequences(notes, n_vocab)  # This function needs to be defined to preprocess your MIDI data

    # Adversarial ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random batch of note sequences
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_seqs = X_train[idx]

        # Generate a batch of latent vectors using the VAE's encoder
        # Only use the third output (z)
        latent_vectors = vae_encoder.predict(real_seqs)[2]

        # Generate a batch of new note sequences using the GAN's generator
        gen_seqs = gan.generator.predict(latent_vectors)

        # Train the discriminator
        d_loss_real = gan.discriminator.train_on_batch(real_seqs, real)
        d_loss_fake = gan.discriminator.train_on_batch(gen_seqs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.combined.train_on_batch(latent_vectors, real)

        # Print the progress and save generated samples at specified intervals
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            # Optionally, save the model and generated samples here


# Create an instance of the GAN class
gan_instance = GAN(rows=sequence_length)

# Train the GAN using the VAE's encoder
train_gan_with_vae(gan_instance, vae_encoder, epochs=10, batch_size=32, sample_interval=10)


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from music21 import instrument, note, chord, stream

# Load the trained models
generator = load_model('/kaggle/working/tschaigenerator_model.h5')
encoder = vae_encoder

def generate_latent_vectors(encoder, num_samples, sequence_length):
    # Generate random sequences as input for the encoder
    random_sequences = np.random.normal(0, 1, (num_samples, sequence_length, 1))
    # Predict the latent vectors
    latent_vectors = encoder.predict(random_sequences)[2]
    return latent_vectors

def generate_music(generator, latent_vectors, int_to_note, num_notes=100):
    # Generate new sequences
    generated_sequences = generator.predict(latent_vectors)
    
    # Convert sequences to notes
    generated_notes = []
    for seq in generated_sequences:
        seq_notes = []
        for note_value in seq:
            rounded_note = int(np.round(note_value))
            # Ensure the note is within the valid range of notes
            if rounded_note in int_to_note:
                seq_notes.append(int_to_note[rounded_note])
            else:
                # Handle out-of-range notes, e.g., by using a default note or ignoring them
                seq_notes.append('C5')  # Example: default to 'C5'
        generated_notes.append(seq_notes)
    return generated_notes


def create_midi(prediction_output, filename):
    """ Convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Define your int_to_note mapping here
int_to_note = {number: note for number, note in enumerate(sorted(set(item for item in notes)))}

# Generate latent vectors
latent_vectors = generate_latent_vectors(encoder, num_samples=10, sequence_length=100)

# Generate music sequences
generated_notes = generate_music(generator, latent_vectors, int_to_note)

# Create MIDI files from the generated sequences
for i, notes in enumerate(generated_notes):
    create_midi(notes, 'tschai_generated_music_{}'.format(i))
